<img src="https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/img/logos_combined_c4i_amds.png?raw=1" alt="Logos Center for Critical Care Computational Intelligence / Amsterdam Medical Data Science" width=256px>

# AmsterdamUMCdb - Freely Accessible ICU Database

version 1.0.2 March 2020  
Copyright &copy; 2003-2022 Amsterdam UMC - Amsterdam Medical Data Science

## Reason for admission
The reason for admission is the primary diagnosis leading to ICU admission. In AmsterdamUMCdb different classification systems were used to document the reason for admission.

### Reason for admission diagnosis systems
- an internally used diagnosis list, that shows similarity with the APACHE II admission diagnosis ('legacy' diagnosis system)
- APACHE II admission (primary) diagnosis
- APACHE IV admission diagnosis
- NICE, the Foundation Dutch Intensive Care Evaluation (Stichting NICE), a national registry which uses APACHE II and IV for primary and secondary diagnosis

In addition, reason for admission has been documented as a free text description for many patients as well, but this is currently not available in the freely accessible version due to re-identification risks.

### Use cases
- Differentiating between surgical and medical admissions
- Selecting specific types of patients (e.g. trauma, cardiac arrest) e.g. 'cohorts'.

### Specific categories
#### Sepsis
Sepsis is a common reason for ICU admission. However, even though all reason for admission coding systems have a sepsis category, it has often not been documented on that category, but instead other conditions were chosen (e.g. sepsis due to intestinal perforation will have the abdominal surgery category selected). In addition, the definition of sepsis has changed over time. Currently, the most recent Sepsis-3 definition has been implemented for AmsterdamUMCdb. You can create the Sepsis-3 cohort by following the code [below](#sepsis3).

## Imports

In [1]:
%matplotlib inline
import amsterdamumcdb as adb
import psycopg2
import sqlalchemy
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl

import io
from IPython.display import display, HTML, Markdown

## Display settings

In [2]:
#matplotlib settings for image size
#needs to be in a different cell from %matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams["figure.dpi"] = 288
plt.rcParams["figure.figsize"] = [16, 12]
plt.rcParams["font.size"] = 12

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 1000

## Connection settings

In [3]:
#Modify config.ini in the root folder of the repository to change the settings to connect to your postgreSQL database
import configparser
import os
config = configparser.ConfigParser()

if os.path.isfile('../../config.ini'):
    config.read('../../config.ini')
else:
    config.read('../../config.SAMPLE.ini')

#Open a connection to the postgres database:
pg_con = psycopg2.connect(database=config['psycopg2']['database'], 
                       user=config['psycopg2']['username'], password=config['psycopg2']['password'], 
                       host=config['psycopg2']['host'], port=config['psycopg2']['port'])
pg_con.set_client_encoding('WIN1252') #Uses code page for Dutch accented characters.
pg_con.set_session(autocommit=True)

cursor = pg_con.cursor()
cursor.execute('SET SCHEMA \'amsterdamumcdb\''); #set search_path to amsterdamumcdb schema

# create SQLAlchemy engine for official pandas database support
engine = sqlalchemy.create_engine('postgresql+psycopg2://', creator=lambda: pg_con)
con = engine.connect()

In [4]:
#loads the admissions table for stratification on admission unit 
sql = """
SELECT * FROM admissions;
"""
admissions = pd.read_sql(sql,con)

In [5]:
#No. patients
no_patients_total = admissions.patientid.nunique()
no_patients_ICU = admissions[admissions['location'].str.contains('IC')].patientid.nunique()
no_patients_MCU = admissions[admissions['location'].str.contains('^MC$')].patientid.nunique()

#No. admissions
no_admissions_total = admissions.admissionid.nunique()
no_admissions_ICU = admissions[admissions['location'].str.contains('IC')].admissionid.nunique()
no_admissions_MCU = admissions[admissions['location'].str.contains('^MC$')].admissionid.nunique()

## Admission types
- Surgical
- Medical
- Urgent
- Elective

## Admissions with reasons for admission
Creates a combined_diagnoses dataframe containing the admissions table joined with the diagnoses from the listitems table. The SQL query can be found [here](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/amsterdamumcdb/sql/diagnosis/reason_for_admission.sql).

In [6]:
combined_diagnoses = adb.get_reason_for_admission(con)
combined_diagnoses.head()

,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,destination,gender,agegroup,dateofdeath,weightgroup,weightsource,heightgroup,heightsource,specialty,diagnosis_type,diagnosis,diagnosis_id,diagnosis_subgroup,diagnosis_subgroup_id,diagnosis_group,diagnosis_group_id,surgical,sepsis_at_admission,sepsis_antibiotics_bool,sepsis_antibiotics_given,other_antibiotics_bool,other_antibiotics_given,sepsis_cultures_bool,sepsis_cultures_drawn
0,0,0,1,IC,0,None,0,2003-2009,148800000,42,16,Vrouw,80+,NaN,60-69,Anamnestisch,160-169,Anamnestisch,Cardiochirurgie,Legacy ICU,CABG,1.0,CABG en Klepchirurgie,1.0,Thoraxchirurgie,1.0,1.0,NaN,NaN,None,1.0,Cefazoline (Kefzol),NaN,None
1,1,1,1,IC,0,None,0,2010-2016,96120000,26,15,Man,60-69,NaN,70-79,Anamnestisch,170-179,Anamnestisch,Cardiochirurgie,Legacy ICU,Bentall,5.0,Aorta chirurgie,3.0,Thoraxchirurgie,1.0,1.0,0.0,NaN,None,1.0,Cefazoline (Kefzol); Vancomycine,NaN,None
2,2,2,1,IC,1,Eerste Hulp afdeling zelfde ziekenhuis,0,2010-2016,84240000,23,15,Man,60-69,NaN,90-99,Anamnestisch,180-189,Anamnestisch,Cardiochirurgie,NICE APACHE IV,CABG with aortic valve replacement,237.0,CABG en Klepchirurgie,1.0,Post-operative cardiovascular,237.0,1.0,0.0,NaN,None,1.0,Cefazoline (Kefzol); Vancomycine,NaN,None
3,3,3,1,IC,0,None,0,2003-2009,84900000,23,14,Man,50-59,NaN,90-99,None,180-189,Gemeten,Cardiochirurgie,None,None,NaN,None,NaN,None,NaN,NaN,NaN,NaN,None,1.0,Cefazoline (Kefzol),NaN,None
4,4,4,1,IC&MC,0,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,180900000,50,19,Man,70-79,NaN,70-79,Anamnestisch,170-179,Anamnestisch,Cardiochirurgie,NICE APACHE IV,"CABG alone, coronary artery bypass grafting",233.0,CABG en Klepchirurgie,1.0,Post-operative cardiovascular,233.0,1.0,0.0,NaN,None,1.0,Cefazoline (Kefzol),NaN,None


In [7]:
#surgical
surgical_total = len(combined_diagnoses[combined_diagnoses['surgical'] == 1])
surgical_pct_total = 100*surgical_total/no_admissions_total

surgical_ICU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 1)  & combined_diagnoses['location'].str.contains('IC')])
surgical_pct_ICU = 100*surgical_ICU/no_admissions_ICU

surgical_MCU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 1)  & combined_diagnoses['location'].str.contains('^MC$')])
surgical_pct_MCU = 100*surgical_MCU/no_admissions_MCU

#medical
medical_total = len(combined_diagnoses[combined_diagnoses['surgical'] == 0])
medical_pct_total = 100*medical_total/no_admissions_total

medical_ICU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 0)  & combined_diagnoses['location'].str.contains('IC')])
medical_pct_ICU = 100*medical_ICU/no_admissions_ICU

medical_MCU = len(combined_diagnoses[(combined_diagnoses['surgical'] == 0)  & combined_diagnoses['location'].str.contains('^MC$')])
medical_pct_MCU = 100*medical_MCU/no_admissions_MCU

#surgical/medical unknown
medical_surgical_na_total = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical'])])
medical_surgical_na_pct_total = 100*medical_surgical_na_total/no_admissions_total

medical_surgical_na_ICU = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical']) & combined_diagnoses['location'].str.contains('IC')])
medical_surgical_na_pct_ICU = 100*medical_surgical_na_ICU/no_admissions_ICU

medical_surgical_na_MCU = len(combined_diagnoses[pd.isnull(combined_diagnoses['surgical']) & combined_diagnoses['location'].str.contains('^MC$')])
medical_surgical_na_pct_MCU = 100*medical_surgical_na_MCU/no_admissions_MCU

#urgency
urgency_total = len(combined_diagnoses[combined_diagnoses['urgency'] == '1']) #apparently the postgresql bit become an object
urgency_pct_total = 100*urgency_total/no_admissions_total

urgency_ICU = len(combined_diagnoses[(combined_diagnoses['urgency'] == '1')  & combined_diagnoses['location'].str.contains('IC')])
urgency_pct_ICU = 100*urgency_ICU/no_admissions_ICU

urgency_MCU = len(combined_diagnoses[(combined_diagnoses['urgency'] == '1')  & combined_diagnoses['location'].str.contains('^MC$')])
urgency_pct_MCU = 100*urgency_MCU/no_admissions_MCU

## Reason for admission based on Legacy/APACHE II or IV diagnosis

## Admission diagnoses
Uses regular expression pattern matching for classification. The regular expressions and cohort definitions can be found [here](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/amsterdamumcdb/cohorts.py).

### Surgical

In [8]:
surgical_patients = combined_diagnoses[(combined_diagnoses['surgical'] == 1)]

surgical_patients_counts = surgical_patients.groupby(
    ['diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
surgical_patients_counts.head()

,diagnosis,n_admissions
0,"CABG alone, coronary artery bypass grafting",1534
1,CABG,1389
2,Diagnose anders,484
3,AVR,458
4,Aortic valve replacement (isolated),431


In [9]:
surgical_patients_counts['n_admissions'].sum()

11293

## medical

In [10]:
medical_patients = combined_diagnoses[(combined_diagnoses['surgical'] == 0)]
medical_patients_counts = medical_patients.groupby(
    ['diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
medical_patients_counts.head()

,diagnosis,n_admissions
0,Cardiac arrest (with or without respiratory arrest;,514
1,Na reanimatie,439
2,Respiratoire insufficiëntie,327
3,Diagnose anders,280
4,"Pneumonia, bacterial",204


In [11]:
medical_patients_counts['n_admissions'].sum()

5819

### Cardiothoracic surgery

In [12]:
cardiosurg = adb.get_cardiothoracic_surgery_patients(con)

cardiosurg_counts = cardiosurg.groupby(
    ['diagnosis'], dropna=False).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
cardiosurg_counts.head()

,diagnosis,n_admissions
0,"CABG alone, coronary artery bypass grafting",1534
1,CABG,1389
2,AVR,458
3,Aortic valve replacement (isolated),431
4,Gecombineerde hartchirurgie,366


In [13]:
cardiosurg_counts['n_admissions'].sum()

5938

### Respiratory failure
This group contains patients with a primary reason of admission of respiratory failure, airway obstructie and pulmonary disorders requiring ICU admission.

In [14]:
respfailure = adb.get_respiratory_failure_patients(con)

respfailure_counts = respfailure.groupby(
    ['diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
respfailure_counts.head()

,diagnosis,n_admissions
0,Respiratoire insufficiëntie,327
1,"Pneumonia, bacterial",204
2,Pneumonie,147
3,"Respiratory- medical, other",116
4,"Obstruction-airway (acute epiglottitis, post-extubation edema, foreign body, etc.)",69


In [15]:
respfailure_counts['n_admissions'].sum()

1517

### Neurosurgery

In [16]:
neurosurg = adb.get_neurosurgery_patients(con)

neurosurg_counts = neurosurg.groupby(
    ['diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
neurosurg_counts.head()

,diagnosis,n_admissions
0,"Neoplasm-cranial, surgery for (excluding transphenoidal)",308
1,Neurochirurgisch Trauma,175
2,TC supra tentorieel,167
3,Overige neurologie/neurochirurgie,106
4,"Hemorrhage/hematoma-intracranial, surgery for",97


In [17]:
neurosurg_counts['n_admissions'].sum()

1618

### Gastro-intestinal surgery

In [18]:
gisurg = adb.get_gastrointestinal_surgery_patients(con)

gisurg_counts = gisurg.groupby(
    ['diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
gisurg_counts.head()

,diagnosis,n_admissions
0,Oesophagus/maag chirurgie,171
1,Dunne/dikkedarm chirurgie (perforatie of obstructie),169
2,"Cancer-esophageal, surgery for (abdominal approach)",110
3,"Cancer-colon/rectal, surgery for (including abdominoperineal resections)",70
4,"Cancer-other GI tract, surgery for (i.e. hepatoma, gallbladder etc.)",67


In [19]:
gisurg_counts['n_admissions'].sum()

1149

### Cardiac arrest

In [20]:
cardiacarrest = adb.get_cardiac_arrest_patients(con)

cardiacarrest_counts = cardiacarrest.groupby(
    ['diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
cardiacarrest_counts.head()

,diagnosis,n_admissions
0,Cardiac arrest (with or without respiratory arrest;,514
1,Na reanimatie,439
2,Cardiac arrest (with or without respiratory arrest; for respiratory arrest see,8


In [21]:
cardiacarrest_counts['n_admissions'].sum()

961

### Vascular surgery

In [22]:
vascsurg = adb.get_vascular_surgery_patients(con)

vascsurg_counts = vascsurg.groupby(
    ['diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
vascsurg_counts.head()

,diagnosis,n_admissions
0,Vaatchirurgie abdominaal,212
1,"Aneurysm, thoracic aortic",94
2,"Aneurysm, abdominal aortic",75
3,Thoraco-abdominale aorta vervanging,68
4,"Aneurysm, thoracic aortic; with dissection",68


In [23]:
vascsurg_counts['n_admissions'].sum()

1037

### Trauma

In [24]:
trauma = adb.get_trauma_patients(con)

trauma_counts = trauma.groupby(['diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
trauma_counts.head()

,diagnosis_group,diagnosis,n_admissions
0,Neurochirurgie,Neurochirurgisch Trauma,175
1,Algemene chirurgie,Multi trauma (operatief),102
2,Algemene chirurgie,Multi trauma (niet operatief),82
3,Non-operative trauma,Head (CNS) only trauma,70
4,Non-operative trauma,Chest/thorax only trauma,64


In [25]:
trauma_counts['n_admissions'].sum()

1242

### <a id=sepsis3></a><a href=#sepsis></a><a id=#sepsis></a><a href=#sepsis></a> <a id=#sepsis></a><a href=#sepsis></a><a id=#sepsis></a><a href=#sepsis></a>  Sepsis
The [Sepsis-3 definition](https://jamanetwork-com.vu-nl.idm.oclc.org/journals/jama/fullarticle/2492881)
requires, in addition to a presumed infection, a Sequential (Sepsis-related) Organ Failure Assessment
(SOFA) score of 2 points or more.

Operationalization:
* presumed infection (with dysregulated host response), based on:
    - sepsis_at_admission flagged in the admission form by the attending physician
    - the admission diagnosis, medical or surgical, is considered a severe infection, e.g., gastro-intestinal perforation, cholangitis, meningitis
    - use of antibiotics that are not used prophylactically after surgery
    - use of antibiotics and cultures drawn within 6 hours of admission

AND

* organ dysfunction based on change in total SOFA score ≥2 points
    - baseline SOFA score assumed to be 0
    - uses data from first 24 hours of ICU admission    """Returns a dataframe containing patients with a diagnosis of sepsis at admission.

In previous versions of the `reason_for_admission.ipynb` only an diagnosis of infection was required to be included in the Sepsis cohort. This group can be re-created using: `amsterdamumcdb.get_infection_patients(con)`. However, since most patients that were admitted for an infection to the ICU already fulfilled the Sepsis-3 definition, the difference is small between these two cohorts: 3152 *vs.* 3019 patients. 

**Note**: calculating SOFA scores require a considerable amount of time.

In [ ]:
sepsis = adb.get_sepsis_patients(con)

sepsis_counts = sepsis.fillna('N/A').groupby(
    ['diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
sepsis_counts.head()

In [ ]:
sepsis_counts['n_admissions'].sum()

### Neurologic disorders

In [ ]:
neuro = adb.get_neuro_patients(con)

neuro_counts = neuro.groupby(
    ['diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
neuro_counts.head()

In [ ]:
neuro_counts['n_admissions'].sum()

### Cardiac disorders

In [ ]:
cardio = adb.get_cardio_patients(con)

cardio_counts = cardio.groupby([
    'diagnosis_group','diagnosis']).size().to_frame('n_admissions').sort_values(
    'n_admissions', ascending=False).reset_index()
cardio_counts.head()

In [ ]:
cardio_counts['n_admissions'].sum()

In [ ]:
#reason for admission

## cardiac surgery
reason_car_surg_total = len(cardiosurg)
reason_car_surg_pct_total = 100*reason_car_surg_total/no_admissions_total
reason_car_surg_ICU = len(cardiosurg[cardiosurg['location'].str.contains('IC')])
reason_car_surg_pct_ICU = 100*reason_car_surg_ICU/no_admissions_ICU
reason_car_surg_MCU = len(cardiosurg[cardiosurg['location'].str.contains('^MC$')])
reason_car_surg_pct_MCU = 100*reason_car_surg_MCU/no_admissions_MCU

## neurosurgery
reason_neu_surg_total = len(neurosurg)
reason_neu_surg_pct_total =  100*reason_neu_surg_total/no_admissions_total
reason_neu_surg_ICU = len(neurosurg[neurosurg['location'].str.contains('IC')])
reason_neu_surg_pct_ICU =  100*reason_neu_surg_ICU/no_admissions_ICU
reason_neu_surg_MCU = len(neurosurg[neurosurg['location'].str.contains('^MC$')])
reason_neu_surg_pct_MCU =  100*reason_neu_surg_MCU/no_admissions_MCU

## gastro-intestinal surgery
reason_gi_surg_total = len(gisurg)
reason_gi_surg_pct_total = 100*reason_gi_surg_total/no_admissions_total
reason_gi_surg_ICU = len(gisurg[gisurg['location'].str.contains('IC')])
reason_gi_surg_pct_ICU = 100*reason_gi_surg_ICU/no_admissions_ICU
reason_gi_surg_MCU = len(gisurg[gisurg['location'].str.contains('^MC$')])
reason_gi_surg_pct_MCU = 100*reason_gi_surg_MCU/no_admissions_MCU

## cardiac arrest
reason_car_arrest_total = len(cardiacarrest)
reason_car_arrest_pct_total = 100*reason_car_arrest_total/no_admissions_total
reason_car_arrest_ICU = len(cardiacarrest[cardiacarrest['location'].str.contains('IC')])
reason_car_arrest_pct_ICU = 100*reason_car_arrest_ICU/no_admissions_ICU
reason_car_arrest_MCU = len(cardiacarrest[cardiacarrest['location'].str.contains('^MC$')])
reason_car_arrest_pct_MCU = 100*reason_car_arrest_MCU/no_admissions_MCU

## vascular surgery
reason_vasc_surg_total = len(vascsurg)
reason_vasc_surg_pct_total = 100*reason_vasc_surg_total/no_admissions_total
reason_vasc_surg_ICU = len(vascsurg[vascsurg['location'].str.contains('IC')])
reason_vasc_surg_pct_ICU = 100*reason_vasc_surg_ICU/no_admissions_ICU
reason_vasc_surg_MCU = len(vascsurg[vascsurg['location'].str.contains('^MC$')])
reason_vasc_surg_pct_MCU = 100*reason_vasc_surg_MCU/no_admissions_MCU

## trauma
reason_trauma_total = len(trauma)
reason_trauma_pct_total = 100*reason_trauma_total/no_admissions_total
reason_trauma_ICU = len(trauma[trauma['location'].str.contains('IC')])
reason_trauma_pct_ICU = 100*reason_trauma_ICU/no_admissions_ICU
reason_trauma_MCU = len(trauma[trauma['location'].str.contains('^MC$')])
reason_trauma_pct_MCU = 100*reason_trauma_MCU/no_admissions_MCU

## sepsis
reason_sepsis_total = len(sepsis)
reason_sepsis_pct_total = 100*reason_sepsis_total/no_admissions_total
reason_sepsis_ICU = len(sepsis[sepsis['location'].str.contains('IC')])
reason_sepsis_pct_ICU = 100*reason_sepsis_ICU/no_admissions_ICU
reason_sepsis_MCU = len(sepsis[sepsis['location'].str.contains('^MC$')])
reason_sepsis_pct_MCU = 100*reason_sepsis_MCU/no_admissions_MCU

## respiratory failure
reason_resp_failure_total = len(respfailure)
reason_resp_failure_pct_total = 100*reason_resp_failure_total/no_admissions_total
reason_resp_failure_ICU = len(respfailure[respfailure['location'].str.contains('IC')])
reason_resp_failure_pct_ICU = 100*reason_resp_failure_ICU/no_admissions_ICU
reason_resp_failure_MCU = len(respfailure[respfailure['location'].str.contains('^MC$')])
reason_resp_failure_pct_MCU = 100*reason_resp_failure_MCU/no_admissions_MCU

## cardio
reason_cardio_total = len(cardio)
reason_cardio_pct_total = 100*reason_cardio_total/no_admissions_total
reason_cardio_ICU = len(cardio[cardio['location'].str.contains('IC')])
reason_cardio_pct_ICU = 100*reason_cardio_ICU/no_admissions_ICU
reason_cardio_MCU = len(cardio[cardio['location'].str.contains('^MC$')])
reason_cardio_pct_MCU = 100*reason_cardio_MCU/no_admissions_MCU

## neuro
reason_neuro_total = len(neuro)
reason_neuro_pct_total = 100*reason_neuro_total/no_admissions_total
reason_neuro_ICU = len(neuro[neuro['location'].str.contains('IC')])
reason_neuro_pct_ICU = 100*reason_neuro_ICU/no_admissions_ICU
reason_neuro_MCU = len(neuro[neuro['location'].str.contains('^MC$')])
reason_neuro_pct_MCU = 100*reason_neuro_MCU/no_admissions_MCU

## Create a table in html

In [ ]:
display(Markdown("""
<table>
    <thead>
        <tr>
            <th style="text-align:left;">Patient Characteristics</th>
            <th style="text-align:right;">Total</th>
            <th style="text-align:right;">ICU</th>
            <th style="text-align:right;">MCU</th>
        </tr>
        <tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align:left">Distinct patients, no.</td>
            <td>{no_patients_total}</td>
            <td>{no_patients_ICU}</td>
            <td>{no_patients_MCU}</td>
        </tr>    
        <tr>
            <td style="text-align:left">ICU admissions, no.</td>
            <td>{no_admissions_total}</td>
            <td>{no_admissions_ICU}</td>
            <td>{no_admissions_MCU}</td>
        </tr>    
        <tr>
            <td style="text-align:left">Surgical admissions, no. (%)</td>
            <td>{surgical_total} ({surgical_pct_total:.2f})</td>
            <td>{surgical_ICU} ({surgical_pct_ICU:.2f})</td>
            <td>{surgical_MCU} ({surgical_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Medical admissions, no. (%)</td>
            <td>{medical_total} ({medical_pct_total:.2f})</td>
            <td>{medical_ICU} ({medical_pct_ICU:.2f})</td>
            <td>{medical_MCU} ({medical_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Surgical/Medical Unknown, no. (%)</td>
            <td>{medical_surgical_na_total} ({medical_surgical_na_pct_total:.2f})</td>
            <td>{medical_surgical_na_ICU} ({medical_surgical_na_pct_ICU:.2f})</td>
            <td>{medical_surgical_na_MCU} ({medical_surgical_na_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Urgent admissions, no. (%)</td>
            <td>{urgency_total} ({urgency_pct_total:.2f})</td>
            <td>{urgency_ICU} ({urgency_pct_ICU:.2f})</td>
            <td>{urgency_MCU} ({urgency_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <th style="text-align:left" colspan=4>Reason for admission</td>
        </tr>
        <tr>
            <td style="text-align:left">Cardiothoracic surgery, no. (%)</td>
            <td>{reason_car_surg_total} ({reason_car_surg_pct_total:.2f})</td>
            <td>{reason_car_surg_ICU} ({reason_car_surg_pct_ICU:.2f})</td>
            <td>{reason_car_surg_MCU} ({reason_car_surg_pct_MCU:.2f})</td>
        </tr>
       <tr>
            <td style="text-align:left">Sepsis, no. (%)</td>
            <td>{reason_sepsis_total} ({reason_sepsis_pct_total:.2f})</td>
            <td>{reason_sepsis_ICU} ({reason_sepsis_pct_ICU:.2f})</td>
            <td>{reason_sepsis_MCU} ({reason_sepsis_pct_MCU:.2f})</td>
        </tr>
         <tr>
            <td style="text-align:left">Respiratory failure, no. (%)</td>
            <td>{reason_resp_failure_total} ({reason_resp_failure_pct_total:.2f})</td>
            <td>{reason_resp_failure_ICU} ({reason_resp_failure_pct_ICU:.2f})</td>
            <td>{reason_resp_failure_MCU} ({reason_resp_failure_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Neurosurgery, no. (%)</td>
            <td>{reason_neu_surg_total} ({reason_neu_surg_pct_total:.2f})</td>
            <td>{reason_neu_surg_ICU} ({reason_neu_surg_pct_ICU:.2f})</td>
            <td>{reason_neu_surg_MCU} ({reason_neu_surg_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Trauma, no. (%)</td>
            <td>{reason_trauma_total} ({reason_trauma_pct_total:.2f})</td>
            <td>{reason_trauma_ICU} ({reason_trauma_pct_ICU:.2f})</td>
            <td>{reason_trauma_MCU} ({reason_trauma_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Gastro-intestinal surgery, no. (%)</td>
            <td>{reason_gi_surg_total} ({reason_gi_surg_pct_total:.2f})</td>
            <td>{reason_gi_surg_ICU} ({reason_gi_surg_pct_ICU:.2f})</td>
            <td>{reason_gi_surg_MCU} ({reason_gi_surg_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Vascular surgery, no. (%)</td>
            <td>{reason_vasc_surg_total} ({reason_vasc_surg_pct_total:.2f})</td>
            <td>{reason_vasc_surg_ICU} ({reason_vasc_surg_pct_ICU:.2f})</td>
            <td>{reason_vasc_surg_MCU} ({reason_vasc_surg_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Cardiac arrest, no. (%)</td>
            <td>{reason_car_arrest_total} ({reason_car_arrest_pct_total:.2f})</td>
            <td>{reason_car_arrest_ICU} ({reason_car_arrest_pct_ICU:.2f})</td>
            <td>{reason_car_arrest_MCU} ({reason_car_arrest_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Neurologic disorders (non-traumatic), no. (%)</td>
            <td>{reason_neuro_total} ({reason_neuro_pct_total:.2f})</td>
            <td>{reason_neuro_ICU} ({reason_neuro_pct_ICU:.2f})</td>
            <td>{reason_neuro_MCU} ({reason_neuro_pct_MCU:.2f})</td>
        </tr>
        <tr>
            <td style="text-align:left">Cardiac disorders (incl. cardiogenic shock) no. (%)</td>
            <td>{reason_cardio_total} ({reason_cardio_pct_total:.2f})</td>
            <td>{reason_cardio_ICU} ({reason_cardio_pct_ICU:.2f})</td>
            <td>{reason_cardio_MCU} ({reason_cardio_pct_MCU:.2f})</td>
        </tr>
    </tbody>
</table>
""".format(
            #No. patients
            no_patients_total = no_patients_total,
            no_patients_ICU = no_patients_ICU,
            no_patients_MCU = no_patients_MCU,

            #No. admissions
            no_admissions_total = no_admissions_total,
            no_admissions_ICU = no_admissions_ICU,
            no_admissions_MCU = no_admissions_MCU,
    
            #surgical
            surgical_total = surgical_total,
            surgical_pct_total = surgical_pct_total,

            surgical_ICU = surgical_ICU,
            surgical_pct_ICU = surgical_pct_ICU,

            surgical_MCU = surgical_MCU,
            surgical_pct_MCU = surgical_pct_MCU,


            #medical
            medical_total = medical_total,
            medical_pct_total = medical_pct_total,

            medical_ICU = medical_ICU,
            medical_pct_ICU = medical_pct_ICU,

            medical_MCU = medical_MCU,
            medical_pct_MCU = medical_pct_MCU,

            #surgical/medical unknown
            medical_surgical_na_total = medical_surgical_na_total,
            medical_surgical_na_pct_total = medical_surgical_na_pct_total,

            medical_surgical_na_ICU = medical_surgical_na_ICU,
            medical_surgical_na_pct_ICU = medical_surgical_na_pct_ICU,

            medical_surgical_na_MCU = medical_surgical_na_MCU,
            medical_surgical_na_pct_MCU = medical_surgical_na_pct_MCU,

            #urgency
            urgency_total = urgency_total,
            urgency_pct_total = urgency_pct_total,

            urgency_ICU = urgency_ICU,
            urgency_pct_ICU = urgency_pct_ICU,

            urgency_MCU = urgency_MCU,
            urgency_pct_MCU = urgency_pct_MCU,

            #reason for admission

            ## cardiac surgery
            reason_car_surg_total = reason_car_surg_total,
            reason_car_surg_pct_total = reason_car_surg_pct_total,
            reason_car_surg_ICU = reason_car_surg_ICU,
            reason_car_surg_pct_ICU = reason_car_surg_pct_ICU,
            reason_car_surg_MCU = reason_car_surg_MCU,
            reason_car_surg_pct_MCU = reason_car_surg_pct_MCU,

            ## neurosurgery
            reason_neu_surg_total = reason_neu_surg_total,
            reason_neu_surg_pct_total = reason_neu_surg_pct_total,
            reason_neu_surg_ICU = reason_neu_surg_ICU,
            reason_neu_surg_pct_ICU = reason_neu_surg_pct_ICU,
            reason_neu_surg_MCU = reason_neu_surg_MCU,
            reason_neu_surg_pct_MCU = reason_neu_surg_pct_MCU,

            ## gastro-intestinal surgery
            reason_gi_surg_total = reason_gi_surg_total,
            reason_gi_surg_pct_total = reason_gi_surg_pct_total,
            reason_gi_surg_ICU = reason_gi_surg_ICU,
            reason_gi_surg_pct_ICU = reason_gi_surg_pct_ICU,
            reason_gi_surg_MCU = reason_gi_surg_MCU,
            reason_gi_surg_pct_MCU = reason_gi_surg_pct_MCU,

            ## cardiac arrest
            reason_car_arrest_total = reason_car_arrest_total,
            reason_car_arrest_pct_total = reason_car_arrest_pct_total,
            reason_car_arrest_ICU = reason_car_arrest_ICU,
            reason_car_arrest_pct_ICU = reason_car_arrest_pct_ICU,
            reason_car_arrest_MCU = reason_car_arrest_MCU,
            reason_car_arrest_pct_MCU = reason_car_arrest_pct_MCU,

            ## vascular surgery
            reason_vasc_surg_total = reason_vasc_surg_total,
            reason_vasc_surg_pct_total = reason_vasc_surg_pct_total,
            reason_vasc_surg_ICU = reason_vasc_surg_ICU,
            reason_vasc_surg_pct_ICU = reason_vasc_surg_pct_ICU,
            reason_vasc_surg_MCU = reason_vasc_surg_MCU,
            reason_vasc_surg_pct_MCU = reason_vasc_surg_pct_MCU,

            ## trauma
            reason_trauma_total = reason_trauma_total,
            reason_trauma_pct_total = reason_trauma_pct_total,
            reason_trauma_ICU = reason_trauma_ICU,
            reason_trauma_pct_ICU = reason_trauma_pct_ICU,
            reason_trauma_MCU = reason_trauma_MCU,
            reason_trauma_pct_MCU = reason_trauma_pct_MCU,

            ## sepsis
            reason_sepsis_total = reason_sepsis_total,
            reason_sepsis_pct_total = reason_sepsis_pct_total,
            reason_sepsis_ICU = reason_sepsis_ICU,
            reason_sepsis_pct_ICU = reason_sepsis_pct_ICU,
            reason_sepsis_MCU = reason_sepsis_MCU,
            reason_sepsis_pct_MCU = reason_sepsis_pct_MCU,

            ## respiratory failure
            reason_resp_failure_total = reason_resp_failure_total,
            reason_resp_failure_pct_total = reason_resp_failure_pct_total,
            reason_resp_failure_ICU = reason_resp_failure_ICU,
            reason_resp_failure_pct_ICU = reason_resp_failure_pct_ICU,
            reason_resp_failure_MCU = reason_resp_failure_MCU,
            reason_resp_failure_pct_MCU = reason_resp_failure_pct_MCU,
    
            ## neurologic disorders
            reason_neuro_total = reason_neuro_total,
            reason_neuro_pct_total = reason_neuro_pct_total,
            reason_neuro_ICU = reason_neuro_ICU,
            reason_neuro_pct_ICU = reason_neuro_pct_ICU,
            reason_neuro_MCU = reason_neuro_MCU,
            reason_neuro_pct_MCU = reason_neuro_pct_MCU,
    
            ## cardiologic disorders
            reason_cardio_total = reason_cardio_total,
            reason_cardio_pct_total = reason_cardio_pct_total,
            reason_cardio_ICU = reason_cardio_ICU,
            reason_cardio_pct_ICU = reason_cardio_pct_ICU,
            reason_cardio_MCU = reason_cardio_MCU,
            reason_cardio_pct_MCU = reason_cardio_pct_MCU
        )
    )
)